In [27]:
####混合ガウス過程回帰モデル
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [37]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [38]:
####データの生成####
##データとセグメントの設定
#データの設定
seg = 8
T = 5000   
k = 100
Lambda = np.random.gamma(15.0, 1/0.5, T)
w = np.random.poisson(Lambda, T)
w[w <= 3] = 3

#セグメントの生成
pi = np.random.dirichlet(np.repeat(5.0, seg), 1).reshape(-1)
Z = np.random.multinomial(1, pi, T)
z = np.dot(Z, np.arange(seg))
index_z = [np.array(np.where(z==j)[0], dtype="int") for j in range(seg)] 
pit = pi.copy()

In [39]:
##ガウス過程からデータの生成
#多項分布から入力変数を生成
alpha = np.repeat(0.15, k)
theta = np.random.dirichlet(alpha, seg)
data = np.zeros((T, k))
for i in range(T):
    data[i, ] = np.random.multinomial(w[i], theta[z[i], ], 1) / w[i]
thetat = theta.copy()

#カーネル関数の生成
n = np.repeat(0, seg)
K = [j for j in range(seg)]
for j in range(seg):
    n[j] = index_z[j].shape[0]
    K[j] = np.dot(data[index_z[j], ], data[index_z[j], ].T)

In [40]:
#モデルパラメータと応答変数を生成
y = np.repeat(0.0, T)
beta = np.random.normal(0, 0.75, seg)
for j in range(seg):
    y[index_z[j]] = beta[j] + np.random.multivariate_normal(np.repeat(0, n[j]), K[j], 1).reshape(-1)
betat = beta.copy()

In [45]:
####MCMC-EMアルゴリズムで混合ガウス過程回帰モデルを推定####
##パラメータ推定のための関数を定義
#多変量正規分布の条件付き期待値と分散を計算する関数
def cdMVN(mu, Cov, department, U):
    #分散共分散行列のブロック行列を定義
    department = np.array([department])
    index = np.delete(np.arange(Cov.shape[0]), department)
    Cov11 = Cov[department, ][:, department]
    Cov12 = Cov[department, ][:, index]
    Cov21 = Cov[:, department][index, ]
    Cov22 = Cov[index, ][:, index]

    #条件付き分散と条件付き平均を計算
    inv_Cov22 = np.linalg.inv(Cov22)
    CDinv = np.dot(Cov12, inv_Cov22)
    CDmu = mu[:, department] + np.dot(CDinv, (U[:, index] - mu[:, index]).T).T   #条件付き平均
    CDvar = Cov11 - np.dot(np.dot(Cov12, inv_Cov22), Cov21)   #条件付き分散
    return CDmu, CDvar

#多変量正規分布の密度関数
def mvdnorm(u, mu, Cov, k):
    er = U - mu 
    inv_Cov = np.linalg.inv(Cov)
    det_Cov = np.linalg.det(Cov)
    Lho = 1 / (np.power(np.sqrt(2*np.pi), k)*det_Cov) * np.exp(-1/2 * np.sum(np.dot(er, inv_Cov) * er, axis=1))
    return Lho 

In [46]:
##アルゴリズムの設定
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10

In [44]:
##事前分布の設定
#潜在変数の事前分布
alpha = 0.1

#モデルパラメータの事前分布
gamma = 0.0
tau = 100
eta = np.power(0.1, 2)

In [42]:
##パラメータの真値
#潜在変数の真値
pi = pit.copy()
Zi = Z.copy()

#モデルパラメータの真値
theta = thetat.copy()
beta = betat.copy()
K = [j for j in range(seg)]
for j in range(seg):
    K[j] = np.dot(data[index_z[j], ], data[index_z[j], ].T)

In [47]:
####パラメータをサンプリング####
##潜在変数zをサンプリング
index_z




[array([  18,   23,   27,   49,   69,  103,  119,  124,  139,  142,  149,
         184,  211,  212,  218,  237,  253,  259,  268,  279,  282,  330,
         333,  359,  392,  414,  418,  428,  463,  465,  474,  556,  575,
         614,  635,  641,  649,  661,  669,  678,  701,  708,  710,  787,
         806,  822,  842,  863,  873,  883,  889,  912,  915,  947, 1005,
        1035, 1054, 1056, 1086, 1112, 1122, 1129, 1156, 1162, 1164, 1174,
        1176, 1191, 1244, 1252, 1260, 1261, 1268, 1291, 1300, 1311, 1367,
        1429, 1435, 1488, 1505, 1508, 1524, 1536, 1548, 1614, 1630, 1637,
        1642, 1653, 1658, 1661, 1692, 1706, 1756, 1834, 1868, 1887, 1888,
        1892, 1898, 1902, 1934, 1944, 1953, 1955, 1988, 2010, 2029, 2039,
        2088, 2103, 2108, 2123, 2132, 2137, 2141, 2184, 2197, 2199, 2228,
        2230, 2244, 2281, 2286, 2308, 2340, 2362, 2396, 2409, 2488, 2500,
        2523, 2590, 2592, 2619, 2620, 2653, 2671, 2679, 2688, 2708, 2718,
        2774, 2775, 2783, 2816, 2822, 

In [14]:
np.repeat(beta[j], n[j]).shape

(413449,)

In [35]:
theta.shape

(8, 100)